In [15]:
import torch
from matplotlib import pyplot as plt
import numpy as np
from collections import OrderedDict
from pyDOE import lhs
from matplotlib import cm

In [16]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [17]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [18]:
class PINNs():
    def __init__(self, X_u, u, X_f, layers):
        # data
        self.x_u = torch.tensor(
            X_u[:, 0:1], requires_grad=True).float().to(device)
        self.t_u = torch.tensor(
            X_u[:, 1:2], requires_grad=True).float().to(device)
        self.alpha_u = torch.tensor(
            X_u[:, 2:3], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(
            X_f[:, 0:1], requires_grad=True).float().to(device)
        self.t_f = torch.tensor(
            X_f[:, 1:2], requires_grad=True).float().to(device)
        self.alpha_f = torch.tensor(
            X_f[:, 2:3], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)

        self.layers = layers

        # deep neurla networks
        self.dnn = DNN(layers).to(device)

         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )

        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0

    def net_u(self, x, t, alpha):
        u = self.dnn(torch.cat([x, t, alpha], dim=1))
        return u
    
    def net_f(self, x, t, alpha):
        u = self.net_u(x, t, alpha)

        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        f = u_t - alpha * u_xx
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.x_u, self.t_u, self.alpha_u)
        f_pred = self.net_f(self.x_f, self.t_f, self.alpha_f)
        loss_u = torch.mean((self.u - u_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)
        
        loss = loss_u + loss_f
        self.optimizer.zero_grad()
        loss.backward()

        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
            )
        return loss

    def train(self, nIter):
        self.dnn.train()
        for epoch in range(nIter):
            u_pred = self.net_u(self.x_u, self.t_u, self.alpha_u)
            f_pred = self.net_f(self.x_f, self.t_f, self.alpha_f)
            loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 100 == 0:
                print(
                    'It: %d, Loss: %.3e' % 
                    (
                        epoch, 
                        loss.item(), 
                    )
                )
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)

    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        alpha = torch.tensor(X[:, 2:3], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t, alpha)
        f = self.net_f(x, t, alpha)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [19]:
T_i = 1.0
T_t = 0.0
T_b = 0.0

N_u = 1000
N_f = 10000
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 1]

N_t = 100
N_x = 100
N_a = 100
t = np.linspace(0, 1, N_t)
x = np.linspace(-1, 1, N_x)
alpha = np.linspace(0.0, 1, N_a)

X, T, ALPHA = np.meshgrid(x, t, alpha)
X_star = np.hstack(
    (X.flatten()[:,None], T.flatten()[:,None], ALPHA.flatten()[:,None]))

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

 # Left boundary, t=0
xx1 = np.hstack(
    (X[0:1,:,:].flatten()[:,None], 
    T[0:1,:,:].flatten()[:,None], 
    ALPHA[0:1,:,:].flatten()[:,None]))
uu1 = np.ones_like(xx1) * T_i
# Lower boundary, x=-1
xx2 = np.hstack(
    (X[:,0:1,:].flatten()[:,None], 
    T[:,0:1,:].flatten()[:,None], 
    ALPHA[:,0:1,:].flatten()[:,None]))
uu2 = np.ones_like(xx2) * T_b
# Upper boundary, x=1
xx3 = np.hstack(
    (X[:,-1:,:].flatten()[:,None], 
    T[:,-1:,:].flatten()[:,None], 
    ALPHA[:,-1:,:].flatten()[:,None]))
uu3 = np.ones_like(xx3) * T_t

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(3, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

In [20]:
model = PINNs(X_u_train, u_train, X_f_train, layers)

In [21]:
%%time

model.train(10)

It: 0, Loss: 6.222e-01
Iter 100, Loss: 5.01203e-02, Loss_u: 4.67464e-02, Loss_f: 3.37395e-03
Iter 200, Loss: 4.20953e-02, Loss_u: 3.86837e-02, Loss_f: 3.41154e-03
Iter 300, Loss: 3.73478e-02, Loss_u: 3.20363e-02, Loss_f: 5.31150e-03
Iter 400, Loss: 2.90038e-02, Loss_u: 2.39951e-02, Loss_f: 5.00864e-03
Iter 500, Loss: 2.23930e-02, Loss_u: 1.79615e-02, Loss_f: 4.43145e-03
Iter 600, Loss: 1.74639e-02, Loss_u: 1.35255e-02, Loss_f: 3.93842e-03
Iter 700, Loss: 1.44193e-02, Loss_u: 1.09474e-02, Loss_f: 3.47184e-03
Iter 800, Loss: 1.25867e-02, Loss_u: 9.54962e-03, Loss_f: 3.03706e-03
Iter 900, Loss: 1.15702e-02, Loss_u: 8.97706e-03, Loss_f: 2.59316e-03
Iter 1000, Loss: 1.07127e-02, Loss_u: 8.32801e-03, Loss_f: 2.38472e-03
Iter 1100, Loss: 1.00278e-02, Loss_u: 7.64799e-03, Loss_f: 2.37978e-03
Iter 1200, Loss: 9.53245e-03, Loss_u: 7.23666e-03, Loss_f: 2.29579e-03
Iter 1300, Loss: 9.16280e-03, Loss_u: 6.97684e-03, Loss_f: 2.18596e-03
Iter 1400, Loss: 8.90381e-03, Loss_u: 6.83994e-03, Loss_f: 2.06